In [36]:
import requests
import json
import pandas as pd
from time import sleep

In [37]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Umbria
df_er = df_comuni[df_comuni['cod_regione'] == '10']

In [38]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_97241/3372484651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_97241/3372484651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
414,103040800030,ALVIANO,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0030,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
378,103040580270,MARSCIANO,CM,M,NaN,7,10,UMBRIA,304,TERNI,058,0270,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
412,103040800010,ACQUASPARTA,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0010,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...


In [39]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [40]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff'][:1]):
   print(f"\nProcessing record {idx + 1}")
   print(f"URL: {url}")

   response = requests.get(url, headers=headers)
   print(f"Response status code: {response.status_code}")
   
   data = response.json()

   comune = df_er.iloc[idx]['desc']
   provincia = df_er.iloc[idx]['provincia']
   regione = df_er.iloc[idx]['regione']
   url = df_er.iloc[idx]['url']

   print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

   # Start with basic comune info
   comune_info = {
       'comune': comune,
       'provincia': provincia,
       'regione': regione,
       'url': url
   }
   
   # Add candidate totals
   if 'cand' in data:
       for cand in data['cand']:
           nome_completo = cand['nome'] + " " + cand['cogn']
           comune_info[f"{nome_completo}_total"] = cand['voti']
           comune_info[f"{nome_completo}_perc"] = cand['perc']
           print(f"{nome_completo}: {cand['voti']} votes ({cand['perc']}%)")

   comuni_data.append(comune_info)
   
   print(f"Processed {comune}")
   sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0010
Response status code: 200
Processing data for Comune: ASSISI, Provincia: TERNI
STEFANIA PROIETTI: 6096 votes (49,27%)
DONATELLA TESEI: 5986 votes (48,38%)
MARCO RIZZO: 123 votes (0,99%)
MARTINA LEONARDI: 52 votes (0,42%)
GIUSEPPE PAOLONE: 32 votes (0,26%)
ELIA FRANCESCO FIORINI: 25 votes (0,20%)
MORENO PASQUINELLI: 23 votes (0,19%)
FABRIZIO PIGNALBERI: 21 votes (0,17%)
GIUSEPPE TRITTO: 15 votes (0,12%)
Processed ASSISI

Creating DataFrame...

Sample of processed data:
   comune provincia regione  \
0  ASSISI     TERNI  UMBRIA   

                                                 url  STEFANIA PROIETTI_total  \
0  https://elezioni.interno.gov.it/risultati/2024...                     6096   

  STEFANIA PROIETTI_perc  DONATELLA TESEI_total DONATELLA TESEI_perc  \
0                  49,27                   5986                48,38   

   MARCO RIZZO_total MARCO RIZZO_perc  ... 

In [41]:
data['cand']

[{'cogn': 'PROIETTI',
  'nome': 'STEFANIA',
  'a_nome': None,
  'd_nasc': 19750105000000,
  'l_nasc': 'ASSISI (PG)',
  'pos': 7,
  'desc_lis_r': None,
  'img_lis_r': None,
  'voti': 6096,
  'perc': '49,27',
  'tot_vot_lis': 4621,
  'perc_lis': '45,62',
  'liste': [{'pos': 9,
    'desc_lis_c': 'PARTITO DEMOCRATICO',
    'img_lis_c': '7-20241117000000-9-304.png',
    'voti': 2191,
    'perc': '21,63'},
   {'pos': 22,
    'desc_lis_c': 'UMBRIA DOMANI PROIETTI PRESIDENTE',
    'img_lis_c': '7-20241117000000-22-304.png',
    'voti': 1252,
    'perc': '12,36'},
   {'pos': 7,
    'desc_lis_c': 'MOVIMENTO 5 STELLE',
    'img_lis_c': '7-20241117000000-7-304.png',
    'voti': 525,
    'perc': '5,18'},
   {'pos': 16,
    'desc_lis_c': 'ALLEANZA VERDI E SINISTRA',
    'img_lis_c': '7-20241117000000-16-304.png',
    'voti': 283,
    'perc': '2,79'},
   {'pos': 23,
    'desc_lis_c': 'UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE',
    'img_lis_c': '7-20241117000000-23-304.png',
    'voti':

In [42]:
df_data

,comune,provincia,regione,url,STEFANIA PROIETTI_total,STEFANIA PROIETTI_perc,DONATELLA TESEI_total,DONATELLA TESEI_perc,MARCO RIZZO_total,MARCO RIZZO_perc,...,GIUSEPPE PAOLONE_total,GIUSEPPE PAOLONE_perc,ELIA FRANCESCO FIORINI_total,ELIA FRANCESCO FIORINI_perc,MORENO PASQUINELLI_total,MORENO PASQUINELLI_perc,FABRIZIO PIGNALBERI_total,FABRIZIO PIGNALBERI_perc,GIUSEPPE TRITTO_total,GIUSEPPE TRITTO_perc
0,ASSISI,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,6096,"49,27",5986,"48,38",123,"0,99",...,32,"0,26",25,"0,20",23,"0,19",21,"0,17",15,"0,12"


In [43]:
# Get all columns containing '_total'
total_columns = [col for col in df_data.columns if '_total' in col]

# Sum all vote totals for each comune
df_data['total_votes'] = df_data[total_columns].sum(axis=1)

# Calculate the spectrum considering all votes
df_data['vote_spectrum'] = round((df_data['STEFANIA PROIETTI_total'] / df_data['total_votes']) * 100, 2)

In [44]:
# df_data['perc_diff'] = round(((df_data['STEFANIA PROIETTI_total'] - df_data['DONATELLA TESEI_total']) / df_data['DONATELLA TESEI_total']) * 100, 2)

In [45]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,comune,provincia,regione,url,STEFANIA PROIETTI_total,STEFANIA PROIETTI_perc,DONATELLA TESEI_total,DONATELLA TESEI_perc,MARCO RIZZO_total,MARCO RIZZO_perc,...,ELIA FRANCESCO FIORINI_total,ELIA FRANCESCO FIORINI_perc,MORENO PASQUINELLI_total,MORENO PASQUINELLI_perc,FABRIZIO PIGNALBERI_total,FABRIZIO PIGNALBERI_perc,GIUSEPPE TRITTO_total,GIUSEPPE TRITTO_perc,total_votes,vote_spectrum
0,Assisi,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,6096,"49,27",5986,"48,38",123,"0,99",...,25,"0,20",23,"0,19",21,"0,17",15,"0,12",12373,49.27


In [46]:
# Use .str.replace() to handle string replacement more safely
df_data['comune'] = df_data['comune'].str.replace("Citta'", "Città", regex=False)


In [47]:
df_data

,comune,provincia,regione,url,STEFANIA PROIETTI_total,STEFANIA PROIETTI_perc,DONATELLA TESEI_total,DONATELLA TESEI_perc,MARCO RIZZO_total,MARCO RIZZO_perc,...,ELIA FRANCESCO FIORINI_total,ELIA FRANCESCO FIORINI_perc,MORENO PASQUINELLI_total,MORENO PASQUINELLI_perc,FABRIZIO PIGNALBERI_total,FABRIZIO PIGNALBERI_perc,GIUSEPPE TRITTO_total,GIUSEPPE TRITTO_perc,total_votes,vote_spectrum
0,Assisi,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,6096,"49,27",5986,"48,38",123,"0,99",...,25,"0,20",23,"0,19",21,"0,17",15,"0,12",12373,49.27


In [ ]:
df_data.to_csv('../output/risultati_um.csv', index=False, encoding='UTF-8-sig')